# AdventOfCode 2017

#### Arnaud Delaunay

## Utils

In [1]:
import sys
sys.setrecursionlimit(100000)
import numpy as np

def get_inp(day, mode='np'):
    if mode=='np':
        try:
            return np.loadtxt('inputs/day%s' % day)
        except:
            return open('inputs/day%s' % day).read()[:-1]
    return open('inputs/day%s' % day).read()[:-1]
        

outputs = {}

def reload_answers(day):
    outputs[day]['answers'] = outputs[day]['func'](day)
    return outputs[day]['answers']

def add_day_func_to_dict(day_func, day, mode='np', callback=None):
    inp = get_inp(day, mode=mode)
    if callback:
        inp = callback(inp)
    f = lambda day : day_func(inp)
    outputs[day] = {
        'func' : f,
        'answers' : f(day)
    }
    return outputs[day]['answers']

# Day 1

In [2]:
def get_output(inp, i, add):
    return int(inp[i]) if inp[i]==inp[np.mod(i+add,len(inp))] else 0

def day_1(inp):
    S1 = 0
    S2 = 0
    for i in range(len(inp)):
        S1+=get_output(inp,i,1)
        S2+=get_output(inp,i,len(inp)//2)
    print(S1,S2)

In [3]:
add_day_func_to_dict(day_1, '1', mode=0)

1119 1420


# Day 2

In [21]:
def day_2_1(inp):
    return int(np.array(list(map(lambda x : x.max() - x.min(), inp))).sum())

def day_2_2(inp):
    def get_div(row):
        for i, e in enumerate(row):
            for e2 in np.hstack((row[:i],row[i+1:])):
                if e%e2==0:
                    return e//e2
    return int(np.array(list(map(get_div, inp))).sum())

day_2 = lambda inp : (day_2_1(inp),day_2_2(inp))

In [22]:
add_day_func_to_dict(day_2, '2', mode='np')

(45351, 275)

# Day 3

In [23]:
#DAY 3.1
def get_path_to_1(inp):
    k = int(np.sqrt(inp))
    K = k/2 - (1-k%2)
    I = k%2
    p = inp-(k*k)
    return abs(K-min(p-1, k))+abs(K+I-max(p-k-1,0))

# %time get_path_to_1(inp)

#DAY 3.2

#USELESS FINALLY
def get_coordinates(inp):
    if inp == 1:
        return 0,0
    k = int(np.sqrt(inp))
    K = k/2
    p = inp-(k*k)
    add = 1 if p==0 else 0 # if it's a square
    I = (-1)**(k%2+1)*(K-min(p-1,k)-add)
    J = (-1)**(k%2+1)*(K+k%2-max(p-k-1,0)-add)
    return I, J

def get_idx(I,J):
    if (I,J) == (0,0):
        return 1
    sign = np.sign(J-I) if J!=I else -1 #we want sign(0)=-1
    K = max(sign*J, -sign*I)-1
    k = K*2+1-min(0,sign)
    N = K +k**2+1
    N += -sign*I - 1*(min(sign, 0)) if -sign*I<=sign*J else -sign*J + k + 1
    return int(N)

M = {}
def compute_sum(I,J):
    if I==0 and J==0:
        return 1
    if '%d,%d' % (I,J) in M:
        return M['%d,%d' % (I,J)]
    this_idx = get_idx(I,J)
    S = 0
    for i in range(-1,2):
        for j in range(-1,2):
            if get_idx(I+i,J+j)<this_idx:
                if '%d,%d' % (I+i,J+j) not in M:
                    M['%d,%d' % (I+i,J+j)] = compute_sum(I+i,J+j)
                S+=M['%d,%d' % (I+i,J+j)]
    return S

def find_smallest_larger_than(X):
    k = 0
    while(True):
        if compute_sum(k,k)>X:
            sign=1
            if compute_sum(-k,-k)<X:
                sign = -1 
            value = compute_sum(sign*(k-1),sign*k)
            s = 2
            while X>value:
                value = compute_sum(sign*(k-min(s,2*k)),sign*(k-max(s-2*k,0)))
                s+=1
            return value
        k += 1
        
# %time find_smallest_larger_than(10211726171726281982716820)

day_3  = lambda inp : (get_path_to_1(inp), find_smallest_larger_than(inp))
add_day_func_to_dict(day_3, '3', mode='np')

(371.0, 369601)

# DAY 4

In [32]:
from functools import reduce
def check_passphrase(paph):
    L = paph.split(' ')
    return len(L)==len(set(L))
def day_4_1(inp):
    return sum(list(map(check_passphrase,inp)))

#inp = get_inp('4').split('\n')[:-1]
#day_4_1(inp)

def doublecheck_passphrase(paph):
    L = list(map(lambda x : set(x), paph.split(' ')))
    unique = reduce(lambda l, x: l.append(x) or l if x not in l else l, L, [])
    return len(unique)==len(L)

def day_4_2(inp):
    return sum(list(map(doublecheck_passphrase,inp)))

#inp = get_inp('4').split('\n')[:-1]
#day_4_2(inp)

day_4 = lambda inp : (day_4_1(inp), day_4_2(inp))

In [33]:
add_day_func_to_dict(day_4, '4', mode=0, callback=lambda x : x.split('\n'))

(386, 208)

# DAY 5

In [34]:
def get_number_of_steps(this_inp, change, change2_if_condition=1, condition=3):
    inp = this_inp.copy()
    cursor = 0
    L = len(inp)
    step = 0
    while True:
        prev = cursor
        cursor += inp[cursor]
        inp[prev] += change2_if_condition if inp[prev]>=condition else change
        step +=1
        if cursor >= L or cursor <0:
            break
    return step

day_5 = lambda inp : (get_number_of_steps(inp, 1), get_number_of_steps(inp, 1, -1, 3))

In [35]:
add_day_func_to_dict(day_5, '5', callback=lambda array : array.astype(int))

(396086, 28675390)

# DAY 6

In [2]:
import multiprocessing
from functools import partial

def day_6(inp):
    L=len(inp)
    states =[] 

    def choose_bank(inp):
        return inp.argmax()

    def get_spread(idxn_b, iv):
        i, v = iv
        idx, n_b = idxn_b
        # print i, v, idx, n_b
        if i == idx:
            v = 0
        S=v+n_b//L
        if np.mod((i-idx)-1, L)<(n_b%L):
            S+=1
        return S

    def spread_blocks(idx, inp):
        n_b = inp[idx] 
        #pool = multiprocessing.Pool()
        #inp = pool.map(partial(get_spread, (idx,n_b)), enumerate(inp))
        inp = list(map(partial(get_spread, (idx,n_b)), enumerate(inp)))
        return np.array(inp) 

    step = 0
    while True:
        idx = choose_bank(inp) 
        inp = spread_blocks(idx, inp)
        hashinp = hash(inp.tostring())
        step += 1
        if hashinp in states:
            break
        states.append(hashinp)

    return (step, len(states)-states.index(hashinp))

In [3]:
add_day_func_to_dict(day_6, '6', mode='np', callback=lambda x : x.astype(int))

(12841, 8038)

# DAY 7

In [13]:
def get_root(inp):
    inp = list(map(lambda x : x.split(' '), inp))
    leafs = dict([(l[0], {'w' : int(l[1].replace('(','').replace(')',''))}) for l in inp if '->' not in l])
    roots = dict([
        (l[0], {
                'w' : int(l[1].replace('(','').replace(')','')),
                'sons' : list(map(lambda x : x.replace(',',''), l[l.index('->')+1:]))
               }) for l in inp if '->' in l])
    leafs_of_roots = [leaf for word in roots for leaf in roots[word]['sons']]
    for root in roots:
        if root not in leafs_of_roots:
            return root, roots, leafs

answer_7_1 = lambda inp : get_root(inp)[0]

def get_weights_of_sons(word, leafs, roots):
    if word in leafs:
        return leafs[word]['w']
    else:
        return np.sum([get_weights_of_sons(w, leafs, roots) for w in roots[word]['sons']]) + roots[word]['w']

def get_unbalanced_son(word, leafs, roots):
    if word in leafs:
        return None, None, None
    L = [get_weights_of_sons(w, leafs, roots) for w in roots[word]['sons']]
    value = len(set(L))==1
    if value:
        return None, None, None
    else:
        for i, l in enumerate(L):
            if L.count(l)==1:
                return roots[word]['sons'][i], l, L[np.mod((i+1),len(L))]

def answer_7_2(inp):
    word, roots, leafs = get_root(inp)
    should_be = 0
    v = 0
    while word is not None:
        prev_word, diff = (word, v-should_be)
        word, should_be, v = get_unbalanced_son(word, leafs, roots)
    #print("Bad value is %d, should be %d (weight of word %s)" % (roots[prev_word]['w'], roots[prev_word]['w'] + diff, prev_word))
    return roots[prev_word]['w'] + diff

day_7 = lambda inp : (answer_7_1(inp), answer_7_2(inp))

In [14]:
add_day_func_to_dict(day_7, '7', mode=0, callback=lambda x : x.split('\n'))

('eugwuhl', 420)

# DAY 8

In [17]:
import operator
import numpy as np

def day_8(inp):
    ACTIONS = {'inc' : 1, 'dec' : -1}
    CONDITIONS = {"==" : lambda x,y : x==y,
                  "!=" : lambda x,y : x!=y,
                  "<=" : lambda x,y : x<=y,
                  ">=" : lambda x,y : x>=y,
                  ">" : lambda x,y : x>y,
                  "<" : lambda x,y : x<y
                 }
    variables = {'max_value' : -np.inf}
    
    def code_parser(line):
        """
        <inp_variable> <action> <var_value> if <cond_variable> <condition> <cond_value>
        """
        inp_variable, action, var_value, _, cond_variable, condition, cond_value = line.split()
        variables[inp_variable] = 0 if inp_variable not in variables else variables[inp_variable]
        variables[cond_variable] = 0 if cond_variable not in variables else variables[cond_variable]
        if CONDITIONS[condition](variables[cond_variable], int(cond_value)):
            variables[inp_variable] = variables[inp_variable] + ACTIONS[action]*int(var_value)
            variables['max_value'] = max(variables['max_value'], variables[inp_variable])

    list(map(code_parser, inp))
    max_value = variables['max_value']
    del variables['max_value']
    return(max(iter(variables.values())), max_value)

In [18]:
add_day_func_to_dict(day_8, '8', mode=0, callback=lambda x : x.split('\n'))

(4567, 5636)

# DAY 9

In [19]:
import re
def remove_char(inp):
    return re.sub('!.{1}', '', inp)

def remove_garbage(inp):
    return re.sub('<.*?>', '', inp)

def count_in_garbage(inp):
    myre = re.compile(r'<.*?>')
    return np.sum(len(x) for x in map(lambda x : x[1:-1], myre.findall(inp)))

def get_subgroups(inp):
    return inp[1:-1].split(',')

def parenthetic_contents(string):
    stack = []
    for i, c in enumerate(string):
        if c == '{':
            stack.append(i)
        elif c == '}' and stack:
            start = stack.pop()
            yield (len(stack), string[start + 1: i])
            
day_9_1 = lambda inp : np.sum([depth+1 for depth, val in parenthetic_contents(remove_garbage(remove_char(inp)))])
day_9_2 = lambda inp : count_in_garbage(remove_char(inp))

day_9 = lambda inp : (day_9_1(inp), day_9_2(inp))

In [20]:
add_day_func_to_dict(day_9, '9', mode=0)

(14204, 6622)

# DAY 10

In [42]:
# DAY 10.1
def modify_list(L, current_position, length, skip_size):
    N = (current_position+length)
    number_of_first_elements = max(0,N-len(L))
    sub_list = L[current_position:N] + L[:number_of_first_elements]
    sub_list.reverse()
    L[current_position:N] = sub_list[:(length-number_of_first_elements)]
    L[:number_of_first_elements] = sub_list[length-number_of_first_elements:]
    return L, np.mod((current_position+length+skip_size),len(L)), skip_size+1

def day_10_1(inp):
    L = list(range(256))
    current_position = 0
    skip_size = 0
    for length in inp:
        L, current_position, skip_size = modify_list(L, current_position, length, skip_size)
    return L[0]*L[1]

In [43]:
inp = get_inp('10')[:-1]

def to_ascii(string):
    return [ord(c) for c in string]

def get_lengths(string):
    return to_ascii(string)+[17, 31, 73, 47, 23]

def get_sparse_hash(string):
    L = list(range(256))
    current_position = 0
    skip_size = 0
    lengths = get_lengths(string)
    for length in lengths*64:
        L, current_position, skip_size = modify_list(L, current_position, length, skip_size)
    return L

def perform_xor(list_of_nums):
    S = 0
    for x in list_of_nums:
        S ^= x
    return S

def to_dense(sparse):
    return ''.join(
        map(
            lambda x : hex(x)[2:] if len(hex(x)[2:])==2 else '0%s' % hex(x)[2:],
            map(
                lambda sixteens : perform_xor(sixteens),
                [sparse[16*i:16*(i+1)] for i in range(16)]
            )
        ))

def day_10_2(inp):
    inp = ','.join(list(map(str,inp)))
    return to_dense(get_sparse_hash(inp))

def day_10_2b(inp):
    return to_dense(get_sparse_hash(inp))

day_10 = lambda inp : (day_10_1(inp), day_10_2(inp))

In [44]:
add_day_func_to_dict(day_10, '10', mode=0, callback=lambda x : map(int, x.split(',')))

(1935, 'a2582a3a0e66e6e86e3812dcb672a272')

# DAY 11

In [29]:
def day_11(inp):
    moves = {
        's' : (2,0),
        'n' : (-2,0),
        'se' : (1, 1),
        'ne' : (-1,1),
        'sw' : (1,-1),
        'nw' : (-1,-1)
    }

    def add(pos, move):
        return (pos[0]+move[0], pos[1]+ move[1])

    def diff(pos, start):
        return (pos[0]-start[0], pos[1]-start[1])

    def compute_steps(pos):
        if abs(pos[1])>abs(pos[0]):
            s = abs(pos[1])
        else:
            s = abs(pos[1])+(abs(pos[0])-abs(pos[1]))/2
        return s

    pos = (0,0)
    steps = []
    poss = []
    for move in inp:
        pos = add(pos, moves[move])
        poss.append(pos)
        steps.append(compute_steps(pos))
    return (steps[-1], max(steps))

In [30]:
add_day_func_to_dict(day_11, '11', mode=0, callback=lambda x : x.split(','))

(664, 1447)

# DAY 12

In [35]:
def parser(line, neighbors):
    idx, neighs = line.split(' <-> ')
    idx = int(idx)
    neighs = list(map(int, neighs.split(',')))
    if idx not in neighbors:
        neighbors[idx] = set()
    list(map(lambda n : neighbors[idx].add(n), neighs))
    return neighs, neighbors

def get_all_neighs_of_idx(inp, idx, neighbors):
    all_neighs = set([idx])
    direct_neighs, neighbors = parser(inp[idx], neighbors)
    list(map(lambda n : all_neighs.add(n), direct_neighs))
    for new_id in direct_neighs:
        if new_id not in neighbors:
            this_neighs, neighbors = get_all_neighs_of_idx(inp, new_id, neighbors)
            list(map(lambda n : all_neighs.add(n), this_neighs))
    return all_neighs, neighbors

def get_all_groups(inp):
    neighbors = {}
    groups = []
    ids_in_groups = set()
    for idx in range(len(inp)):
        if idx not in ids_in_groups:
            new_group, neighbors = get_all_neighs_of_idx(inp, idx, neighbors)
            ids_in_groups = ids_in_groups.union(new_group)
            groups.append(new_group)
    return groups

""" TESTS
test = ["0 <-> 2", "1 <-> 1","2 <-> 0, 3, 4","3 <-> 2, 4","4 <-> 2, 3, 6","5 <-> 6","6 <-> 4, 5"]group_test, _ = get_all_neighs_of_idx(test, 0, {})
groups = get_all_groups(test)
groups
"""

def day_12(inp):
    group_real, _ = get_all_neighs_of_idx(inp, 0, {})
    groups = get_all_groups(inp)
    return (len(group_real), len(groups))

In [36]:
add_day_func_to_dict(day_12, '12', mode=0, callback=lambda x : x.split('\n'))

(130, 189)

# Day 13

In [45]:
def get_pos_of_scanner(rang, t):
    rest = np.mod(t,2*rang-2)
    return rest if rest<rang else np.mod(-rest+rang-1,rang-1)
    
def get_layers(inp):
    return list(map(lambda l : (int(l.split(': ')[0]), int(l.split(': ')[1])), inp))
    
def is_caught_on_layer(layer, delay=0):
    return (True, layer[0]*layer[1]) if get_pos_of_scanner(layer[1], layer[0]+delay)==0 else (False, 0)

def day_13_1(inp, delay=0):
    layers = get_layers(inp)
    return np.sum(list(map(lambda layer : is_caught_on_layer(layer, delay=delay)[1], layers)))

def day_13_2(inp):
    layers = get_layers(inp)
    delay = 1
    while True:
        caught = False
        for layer in layers:
            caught = is_caught_on_layer(layer, delay=delay)[0]
            if caught:
                break
        if not caught:
            return delay
        delay+=1

day_13 = lambda inp : (day_13_1(inp), day_13_2(inp))

In [46]:
add_day_func_to_dict(day_13, '13', mode=0, callback=lambda x : x.split('\n')[:-1])

(1900, 3966414)

# Day 14

In [49]:
def get_bin(e):
    scale = 32 ## equals to hexadecimal
    num_of_bits = 4
    return bin(int(e, scale))[2:].zfill(num_of_bits)

def to_car(binary):
    return binary.replace('1', '#').replace('0', '.')

def get_matrix(inp):
    L = []
    for i in range(128):
        kh = day_10_2b(inp+'-%d' % i)
        L.append(''.join(list(map(get_bin,kh))))
    return L

def day_14_1(inp):
    return ''.join(get_matrix(inp)).count('1')

def get_neighs(pos, group=set(), L=None):
    i,j = pos
    neighs = set()
    for p,k in [(i+1,j), (i-1,j), (i, j+1), (i, j-1)]:
        if p in range(128) and k in range(128) and (p,k) not in group:
            if L[p][k]=='1':
                neighs.add((p,k))
                group.add((p,k))
    l = len(group)
    size_of_group = l-1
    while len(group)!=size_of_group:
        size_of_group = len(group)
        for neigh in neighs:
            group = get_neighs(neigh, group, L=L)
    return group

def day_14_2(inp):
    L = get_matrix(inp)
    viewed = set()
    groups = []
    for i in range(128):
        for j in range(128):
            if (i,j) not in viewed and L[i][j] != '0':
                group = get_neighs((i,j), L=L)
                viewed = viewed.union(group)
                groups.append(group)
    return len(groups)

day_14 = lambda inp : (day_14_1(inp), day_14_2(inp))

In [50]:
add_day_func_to_dict(day_14, '14', mode=0)

(8140, 1182)

# DAY 15

In [51]:
factor_A = 16807
factor_B = 48271
quotient = 2147483647

In [59]:
def get_next(previous, factor):
    return (previous*factor)%quotient

def get_n_bin_digits(number):
    number = str(bin(number))
    if len(number)<16:
        number = "0"*16+number.replace('b','0')
    return number[-16:]

def generators(inp, factor, stop=40e6, conditions=(1,1)):
    count = 0
    genA = generator(inp[0], factor[0], condition=conditions[0])
    genB = generator(inp[1], factor[1], condition=conditions[1])
    while count<stop:
        count+=1
        yield next(genA)==next(genB)

def generator(inp, factor, condition=1):
    while True:
        inp = get_next(inp, factor)
        while inp%condition!=0:
            inp = get_next(inp, factor)
        yield get_n_bin_digits(inp)
        
def day_15_1(inp):
    return np.sum(generators(inp, (factor_A, factor_B)))

def day_15_2(inp):
    return np.sum(generators(inp, (factor_A, factor_B), stop=5e6, conditions=(4,8)))

day_15 = lambda inp : (day_15_1(inp), day_15_2(inp))

In [61]:
add_day_func_to_dict(day_15, '15', mode='np', callback=lambda array : array.astype(int))

(609, 253)

# DAY 16

In [69]:
def spin(programs, param):
    L = len(programs)
    n = int(param)
    programs = [programs[(i-n)%L] for i in range(L)]
    return programs

def exchange(programs, param):
    A, B = map(int, param.split('/'))
    tmp = programs[A]
    programs[A] = programs[B]
    programs[B] = tmp
    return programs

def partner(programs, param):
    A, B = param.split('/')
    tmp = programs.index(B)
    programs[programs.index(A)]=B
    programs[tmp] = A
    return programs

mapper = {
    's' : spin,
    'p' : partner,
    'x' : exchange
}

def get_transformation(element):
    return lambda programs : mapper[element[0]](programs, element[1:])

In [70]:
def apply_to_prog(transfos, programs):
    for transf in transfos:
        programs = transf(programs)
    return programs

In [82]:
def day_16_1(inp):
    mapper = {
        's' : spin,
        'p' : partner,
        'x' : exchange
    }

    def get_transformation(element):
        return lambda programs : mapper[element[0]](programs, element[1:])
    
    programs = [str(chr(i+97)) for i in range(16)]
    transfos = list(map(get_transformation, inp.split(',')))
    return ''.join(apply_to_prog(transfos, programs))

In [83]:
def get_state(programs):
    return int(''.join(list(map(lambda x : str(ord(x)),programs))))

In [86]:
def day_16_2(inp):
    programs = [str(chr(i+97)) for i in range(16)]
    transformations = list(map(get_transformation, inp.split(',')))
    states = set()
    list_of_status = []
    for tour in range(1000):
        programs = apply_to_prog(transformations, programs)
        state = get_state(programs)
        list_of_status.append(''.join(programs))
        if state in states:
            print('Found a loop with state : %s for tour %d' %  (''.join(programs),tour))
            break
        else:
            states.add(state)

    return list_of_status[int(1e9%tour)-1]

day_16 = lambda inp : (day_16_1(inp), day_16_2(inp))

In [87]:
add_day_func_to_dict(day_16, '16', mode='0')

Found a loop with state : doeaimlbnpjchfkg for tour 30


('doeaimlbnpjchfkg', 'agndefjhibklmocp')

# Day 17

In [4]:
inp = get_inp('17').astype(int)
inp

array(382)

In [37]:
def day_17_1(inp):
    L = [0]
    current = 0
    for i in range(1, 2018):
        current = (current+inp+1)%(len(L))
        L.insert(current,i)
    return L[current+1]

def day_17_2(inp):
    current = 0
    last_insert = 0
    for i in range(1, 50000000):
        current = (current+inp+1)%i
        last_insert = i if current==0 else last_insert
    return last_insert
day_17 = lambda inp : (day_17_1(inp), day_17_2(inp))

In [38]:
add_day_func_to_dict(day_17, '17', mode='np', callback=lambda x : x.astype(int))

(1561, 33454823)

# Day 18

In [2]:
from parsy import regex, string, seq

def get_parser():
    command = regex('[a-z]{3}')
    space = string(" ")
    var = regex('[a-z]{1}')
    value = (regex('[-+]?\d+').map(int) | var).optional()
    action = seq(command, space >> value, space.optional() >> value)
    return action


In [3]:
inp_test = ['snd 1','snd 2','snd p','rcv a','rcv b','rcv c','rcv d']

import socket

def create_socket(p):
    serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    host = socket.gethostname()
    port = 9995+p
    serversocket.bind((host, port))                                  
    serversocket.listen(10000)
    return serversocket

def send_to_socket(value, p):
    new_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    new_sock.connect((socket.gethostname(),9995+p))
    new_sock.send(str(value).encode('ascii'))
    new_sock.close()
    
def get_value_from_socket_queue(p, sockets):
    clientsocket, addr = sockets[p].accept()
    value = int(clientsocket.recv(1024).decode('ascii'))
    clientsocket.close()
    return value                    
   
sockets = [create_socket(p) for p in range(2)]

variables = {
    2 : {
        'played' : 0
    },
    0 : {
        'p' : 0,
    },
    1 : {
        'p' : 1,
        'send' : 0
    }
}

def check(var, p=0):
    if var not in variables[p]:
        variables[p][var] = 0
    
def get_value(value, p=0):
    if type(value)==int:
        return value
    else:
        check(value, p=p)
        return variables[p][value]
    
def snd_1(var, value, p=2):
    check(var)
    variables[p]['played'] = variables[p][var]
    return 1, None
    
def snd(var, value, p=0):
    v = get_value(var, p=p)
    if p==1:
        variables[p]['send']+=1
        with open('out.txt', 'a') as out:
            out.write('%d\n' % variables[p]['send'])
    send_to_socket(v, (1-p))
    return 1, None
    
def set_(var, value, p=0):
    variables[p][var] = get_value(value, p=p)
    return 1, None
    
def add(var, value, p=0):
    check(var, p=p)
    variables[p][var] += get_value(value, p=p)
    return 1, None
    
def mul(var, value, p=0):
    check(var, p=p)
    variables[p][var] *= get_value(value, p=p)
    return 1, None
    
def mod(var, value, p=0):
    check(var, p=p)
    variables[p][var] %= get_value(value, p=p)
    return 1, None

def rcv(var, value, p=0):
    value = get_value_from_socket_queue(p, sockets)
    variables[p][var] = value
    return 1, None

def rcv_1(var, value, p=2):
    check(var)
    v = None
    if variables[p][var]!=0:
        v = variables[p]['played']
    return 1, v
    
def jgz(var, value, p=0):
    next_step=1
    if get_value(var, p=p)>0:
        next_step = get_value(value, p=p)
    return next_step, None

mapper = {
    'snd' : snd,
    'set' : set_,
    'add' : add,
    'mul' : mul,
    'mod' : mod,
    'rcv' : rcv,
    'jgz' : jgz
}
mapper_1 = mapper.copy()
mapper_1['snd'] = snd_1
mapper_1['rcv'] = rcv_1

action = get_parser()
def apply_func(inp, idx, p, mapper=mapper, action=action):
    #print("[%d]%s (%s)" % (p, inp[idx], variables[p]))
    parsed = action.parse(inp[idx])
    return mapper[parsed[0]](parsed[1], parsed[2], p=p)

def day_18_1(inp):
    idx = 0
    v = None
    while True:
        if v is not None:
            break
        next_step, v = apply_func(inp, idx, 2, mapper=mapper_1)
        idx += next_step
    return v

import threading
import time


class ThreadingWorker(object):

    def __init__(self, actions, p=0):
        self.p = p
        self.actions = actions
        self.idx = 0
        self.thread = threading.Thread(target=self.run, args=())
        self.thread.daemon = True                            # Daemonize thread
        self.thread.start()                                  # Start the execution

    def run(self):
        while True:
            step, value = apply_func(self.actions, self.idx, self.p)
            self.idx += step
            
def day_18_2(inp):
    thread0 = ThreadingWorker(inp, p=0)
    thread1 = ThreadingWorker(inp, p=1)
    v=0
    time.sleep(5)
    while True:
        with open('out.txt') as out:
            new_v = int(out.readlines()[-1][:-1])
        if new_v==v:
            break
        v=new_v
        time.sleep(5)
    return v


In [4]:
day_18_1(get_inp(18).split('\n'))

2951

In [5]:
day_18_2(get_inp(18).split('\n'))

7366

# Day 19

In [92]:
def get_start(inp):
    return ((1,inp[1].index('|')),[1,0])

def get_next_pos(inp, pos, direction, suite):
    char = inp[pos[0]][pos[1]]
    if char==" ":
        return False, False, suite
    potential_directions = [[1,0], [-1,0],[0,1],[0,-1]]
    del potential_directions[potential_directions.index([-direction[0],-direction[1]])]
    if char=="+":
        for direct in potential_directions:
            new_char = inp[pos[0]+direct[0]][pos[1]+direct[1]]
            if new_char!=" ":
                new_direction = direct
                break
        return ((pos[0]+new_direction[0], pos[1]+new_direction[1]), new_direction, suite)
    if char not in ['|', '-', '+']:
        suite.append(char)
    return ((pos[0]+direction[0], pos[1]+direction[1]), direction, suite)
    
def day_19(inp):
    pos, direction = get_start(inp)
    suite = []
    step=0
    while True:
        pos, direction, suite = get_next_pos(inp, pos, direction, suite)
        step+=1
        if pos==False:
            break
    return("".join(suite), step)

In [93]:
add_day_func_to_dict(day_19, '19', mode='0', callback=lambda x : x[:-1].split('\n'))

('EOCZQMURF', 16312)

# Day 20

In [2]:
inp = get_inp(20).split('\n')

In [180]:
from parsy import regex, string, seq
import pandas as pd

def get_vectors(inp, numpy=True):
    value = regex('[-+]?\d+').map(int)
    parser1 = seq(regex("[a-z]{1}=<") >> value, string(",") >> value, (string(",") >> value).skip(string(">")))
    parser = seq(parser1, string(", ") >> parser1, string(", ") >> parser1)
    func = lambda x : x
    if numpy:
        func = lambda x : np.array(x)
    return list(map(lambda x : func(parser.parse(x)), inp))

def get_manhattan(vector):
    return sum(map(abs,vector))

def day_20_1(inp):
    all_vectors = get_vectors(inp)
    all_manhattans = [list(map(get_manhattan, list_of_v)) for list_of_v in all_vectors]
    df = pd.DataFrame({
        'manhattan_p' : list(map(lambda x : x[0], all_manhattans)),
        'manhattan_v' : list(map(lambda x : x[1], all_manhattans)),
        'manhattan_a' : list(map(lambda x : x[2], all_manhattans))
    }).sort_values(['manhattan_%s' % c for c in ['a','v','p']])
    return df.index[0]

def move(particle):
    particle[1] = list(np.array(particle[1])+np.array(particle[2]))
    particle[0] = list(np.array(particle[0])+np.array(particle[1]))
    return particle

def get_dup(L):
    L_sort = sorted(L)
    L_argsort = sorted(range(len(L)), key=lambda k: L[k])
    dups = []
    dup_args = []
    for i, element in enumerate(L_sort):
        if element in dups:
            dup_args.append(L_argsort[i])
        else:
            if i<(len(L)-1) and element==L_sort[i+1]:
                dups.append(element)
                dup_args.append(L_argsort[i])
    return dup_args

def get_collisions(all_vectors):
    positions = list(map(lambda x : x[0], all_vectors))
    return get_dup(positions)

def remove(vectors):
    collisions = get_collisions(vectors)
    vectors = [vector for i, vector in enumerate(vectors) if i not in collisions]
    return vectors

def iterate(vectors):
    return remove(list(map(move, vectors)))

def day_20_2(inp, iterations=50):
    all_vectors = remove(get_vectors(inp, numpy=False))
    for i in range(iterations):
        all_vectors = iterate(all_vectors)
    return len(all_vectors)

day_20 = lambda inp : (day_20_1(inp), day_20_2(inp))

## Day 20 bis (direct computation)

In [65]:
def get_n(L):
    a, b, c = L
    if a==b and b==c:
        return 'all'
    if c==0:
        return 0
    if a==0:
        if b==0:
            return -np.inf
        else:
            if c%b==0:
                return -c//b
        return -np.inf
    delta = b**2-4*a*c
    if delta<0:
        return -np.inf
    if delta==0:
        return -b//(2*a) if b%(2*a)==0 else -np.inf
    return max((-b-np.sqrt(delta))/(2*a), (-b+np.sqrt(delta))/(2*a))

In [113]:
def will_colisate(arrays,i,j):
    zero = np.zeros(3)
    I = arrays[i]
    J = arrays[j]
    A = I[2]-J[2]
    B = I[1]-J[1]
    C = I[0]-J[0]
    N = list(map(get_n, [[A[i], B[i], C[i]] for i in range(3)]))
    if N==['all']*3:
        return 0
    Nint = [n for n in N if type(n)==np.int64]
    if len(Nint)>0:
        return Nint[0] if (Nint==Nint[0]).all() else -1
    return -1


In [157]:
all_vectors = get_vectors(inp)
M = -1*(np.ones((len(all_vectors), len(all_vectors))))
for i in range(len(all_vectors)):
    for j in range((i+1), len(all_vectors)):
        M[i,j] = will_colisate(all_vectors, i,j)

In [159]:
i = 0
out = set()
while True:
    Mtmp = M[M>i]
    if len(Mtmp)==0:
        break
    i = Mtmp.min()
    for k,p in np.argwhere(M==i):
        if k not in out and p not in out:
            out.add(k)
            out.add(p)

In [160]:
1000-len(out)

482